## run this cell before starting spark cluster connection (corrects module error)

In [49]:
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [50]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [51]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', None)

### select the data

In [52]:
df_ps = spark.read.json("/user/sprenner/training_official_april_hourly_proportion.json")

### Create temporary table view

In [53]:
from pyspark.ml.feature import Bucketizer

splits = [-float("inf"), 2, float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="hits", outputCol="bucketedFeatures")
bucketedData = bucketizer.transform(df_ps)
bucketedData.createOrReplaceTempView("bucket")

In [54]:
spark.sql("SELECT bucketedFeatures, count(*) \
            FROM bucket \
            GROUP BY  bucketedFeatures \
            ORDER BY 1 ASC").toPandas()

bucketedFeatures  count(1)
0               0.0    541829
1               1.0    235457

In [55]:
final_data = bucketedData.withColumn("class_bucket", when(bucketedData["bucketedFeatures"] == 0.0, "nocache").otherwise("cache"))

In [56]:
final_data.createOrReplaceTempView("final")
spark.sql("SELECT class_bucket, count(*) \
            FROM final \
            GROUP BY  class_bucket \
            ORDER BY 1 ASC").toPandas()

class_bucket  count(1)
0        cache    235457
1      nocache    541829

In [57]:
final_data.describe().toPandas()

summary  account           accountHash         clientState  \
0   count   777286                777286              777286   
1    mean     None   1.935684591732567E8                None   
2  stddev     None  1.1361317241087098E9                None   
3     min  aagaard           -2127257360  CannotAuthenticate   
4     max   zzheng            2143592175      SourceNotFound   

       clientStateHash              count  \
0               777286             775662   
1  -9.74016245191591E8  3681.395477669397   
2  2.621298683008777E8  7425.211531293654   
3          -1922357674                  1   
4           1269532611             199996   

                                             dataset           datasetHash  \
0                                             777286                777286   
1                                               None  -1.072936835947129E7   
2                                               None  1.2586751291558852E9   
3  data10_hi.00169927.physics_bulk.ROOT.NTUP_HI.r...           -2146605558   
4  mc16_valid.423105.Pythia8EvtGen_A14NNPDF23LO_g...            2147407155   

  datasetScope      datasetScopeHash dataset_version   dataset_versionHash  \
0       777286                777286          777286                777286   
1         None  -7.552649492442358E8            None  1.3991617429934412E7   
2         None   8.314865501449589E8            None    1.30457085435514E9   
3    data10_hi           -2057006107     c1130_m1847           -2146991447   
4   mc16_valid            2109319881      x557_m1947            2146405934   

  datatype         datatypeHash eventType eventTypeHash eventVersion  \
0   777286               777286    777286        777286       777286   
1     None  4.947060556405943E8      None    5.937808E7         None   
2     None  1.170030329263278E9      None           0.0         None   
3      AOD          -2070622869  download      59378080       1.13.2   
4      log           2124512252  download      59378080       1.15.5   

       eventVersionHash   fileScope                            filename  \
0                777286      777286                              777286   
1  -1.992468944589716E9        None                                None   
2   3.073273964746113E8        None                                None   
3           -2044360925   data10_hi    AOD.05371635._000001.pool.root.1   
4            1261702107  mc16_valid  log.13787275._000072.job.log.tgz.1   

           filenameHash       filesize_double               hits  \
0                777286                777286             777286   
1   -248557.46481861244    6.11402028756827E8  2.010655022732945   
2  1.2402658149710772E9  1.2408771809945774E9  2.337590092629499   
3           -2147478094                 304.0                  1   
4            2147482617       1.1991446515E10                 40   

                   hostname           hostnameHash           hour  \
0                    777286                 777286         777286   
1                      None  -1.1804844919942595E8           None   
2                      None   1.2286452227051136E9           None   
3    ReCaS-ui-01.cs.infn.it            -2133418459  2018-04-01 00   
4  xenia.nevis.columbia.edu             2129595856  2018-04-30 23   

                hourHash                 id       id_proportion  \
0                 777286             777286              579384   
1  -2.5176970566176414E7           388643.5   370425.3149948911   
2   1.2215909376898797E9  224383.2850061846  228391.37208069855   
3            -2138243719                  1                   2   
4             2144044799             777286              777286   

       int_percentage                                     join_key_april  \
0              579384                                             777286   
1    31.2628584841832                                               None   
2  28.916864244093127                                               N

In [58]:
final_data = final_data.withColumn("fileScopeHash", hash(final_data['fileScope']))

In [59]:
final_data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in final_data.columns]).limit(10).toPandas()

account  accountHash  clientState  clientStateHash  count  dataset  \
0        0            0            0                0   1624        0   

   datasetHash  datasetScope  datasetScopeHash  dataset_version  \
0            0             0                 0                0   

   dataset_versionHash  datatype  datatypeHash  eventType  eventTypeHash  \
0                    0         0             0          0              0   

   eventVersion  eventVersionHash  fileScope  filename  filenameHash  \
0             0                 0          0         0             0   

   filesize_double  hits  hostname  hostnameHash  hour  hourHash  id  \
0                0     0         0             0     0         0   0   

   id_proportion  int_percentage  join_key_april  join_key_lookup  prod_step  \
0         197902          197902               0             1624          0   

   prod_stepHash  project  projectHash  protocol  protocolHash  remoteSite  \
0              0        0            0         0             0           0   

   remoteSiteHash  run_number  run_numberHash  scopeHash  stream_name  \
0               0           0               0          0            0   

   stream_nameHash  traceId  traceIdHash  traceIp  traceIpHash  \
0                0        0            0        0            0   

   traceTimeentryUnix  transferStart  uuid  uuidHash  bucketedFeatures  \
0                   0              0     0         0                 0   

   class_bucket  fileScopeHash  
0             0              0

In [60]:
final_data = final_data.fillna(0,['count', 'int_percentage'])

In [61]:
final_data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in final_data.columns]).limit(10).toPandas()

account  accountHash  clientState  clientStateHash  count  dataset  \
0        0            0            0                0      0        0   

   datasetHash  datasetScope  datasetScopeHash  dataset_version  \
0            0             0                 0                0   

   dataset_versionHash  datatype  datatypeHash  eventType  eventTypeHash  \
0                    0         0             0          0              0   

   eventVersion  eventVersionHash  fileScope  filename  filenameHash  \
0             0                 0          0         0             0   

   filesize_double  hits  hostname  hostnameHash  hour  hourHash  id  \
0                0     0         0             0     0         0   0   

   id_proportion  int_percentage  join_key_april  join_key_lookup  prod_step  \
0         197902               0               0             1624          0   

   prod_stepHash  project  projectHash  protocol  protocolHash  remoteSite  \
0              0        0            0         0             0           0   

   remoteSiteHash  run_number  run_numberHash  scopeHash  stream_name  \
0               0           0               0          0            0   

   stream_nameHash  traceId  traceIdHash  traceIp  traceIpHash  \
0                0        0            0        0            0   

   traceTimeentryUnix  transferStart  uuid  uuidHash  bucketedFeatures  \
0                   0              0     0         0                 0   

   class_bucket  fileScopeHash  
0             0              0

## Bucketing and Hashing End

In [62]:
final_data = final_data.orderBy("traceTimeentryUnix", ascending=True)

In [63]:
final_data.createOrReplaceTempView('final')
final_data = spark.sql("SELECT ROW_NUMBER() OVER (ORDER BY traceTimeentryUnix) as idx, * FROM final")

In [66]:
final_data.createOrReplaceTempView('final')
spark.sql("SELECT * FROM final WHERE idx <= 621828").count()

621828

In [67]:
spark.sql("SELECT * FROM final WHERE idx > 621828").count()

155458

In [68]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

assembler = VectorAssembler(inputCols=["accountHash","clientStateHash","count","datasetHash","datasetScopeHash","dataset_versionHash",
                                       "datatypeHash","eventTypeHash","eventVersionHash","filenameHash","filesize_double","hostnameHash",
                                       "hourHash","int_percentage","prod_stepHash","projectHash","protocolHash","remoteSiteHash","run_numberHash",
                                       "stream_nameHash","traceIdHash","traceIpHash","traceTimeentryUnix","transferStart","uuidHash"], 
                            outputCol="features")
output = assembler.transform(final_data)
#output = output.select("bucketedFeatures", "features")

output.createOrReplaceTempView('out')
train_data = spark.sql("SELECT * FROM out WHERE idx <= 621828")
test_data = spark.sql("SELECT * FROM out WHERE idx > 621828")

#train_data, test_data = output.randomSplit([.8,.2],seed=1234)
rf = RandomForestClassifier(labelCol='bucketedFeatures', featuresCol='features',numTrees=300, seed=1234)
model = rf.fit(train_data)
predictions = model.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol="bucketedFeatures", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.0644869


In [17]:
model.featureImportances

SparseVector(25, {0: 0.0211, 1: 0.2591, 2: 0.0197, 3: 0.0016, 4: 0.0052, 5: 0.0025, 6: 0.0426, 8: 0.0009, 9: 0.0, 10: 0.0866, 11: 0.0062, 12: 0.0009, 13: 0.448, 14: 0.0064, 15: 0.0052, 16: 0.0433, 17: 0.0079, 18: 0.0013, 19: 0.0094, 20: 0.0, 21: 0.0173, 22: 0.0058, 23: 0.0063, 24: 0.0026})

In [69]:
predictions = predictions.withColumn("filesize", predictions["filesize_double"].cast(IntegerType()))

In [70]:
final_output = predictions.select('uuid', 'account', 'datasetScope', 'dataset', 'fileScope','filename', 'eventType', 'clientState', 
                                  from_unixtime(predictions.traceTimeentryUnix, format='yyyy-MM-dd').alias('day'), 'traceTimeentryUnix', 
                                  'filesize', 'prediction', 'int_percentage').sort(col("day").asc())

In [71]:
final_output.count()

155458

In [72]:
final_output.limit(10).toPandas()

uuid   account  datasetScope  \
0  8d160961f13a4effbc0a381a6518d5ca  tofitsch    mc15_14TeV   
1  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
2  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
3  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
4  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
5  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
6  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
7  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
8  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
9  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   

                                             dataset     fileScope  \
0  mc15_14TeV.167796.Sherpa_CT10_WtaunuMassiveCBP...    mc15_14TeV   
1  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
2  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
3  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
4  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
5  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
6  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
7  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
8  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
9  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   

                                    filename eventType         clientState  \
0   DAOD_TRUTH3.13422301._000001.pool.root.1  download                DONE   
1  DAOD_HIGG5D1.12819918._000580.pool.root.1  download  ServiceUnavailable   
2  DAOD_HIGG5D1.12819918._000581.pool.root.1  download  ServiceUnavailable   
3  DAOD_HIGG5D1.12819918._000581.pool.root.1  download  ServiceUnavailable   
4  DAOD_HIGG5D1.12819918._000580.pool.root.1  download  ServiceUnavailable   
5  DAOD_HIGG5D1.12819918._000580.pool.root.1  download  ServiceUnavailable   
6  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   
7  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   
8  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   
9  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   

          day  traceTimeentryUnix    filesize  prediction  int_percentage  
0  2018-04-22        1.524413e+09   236600961         0.0             100  
1  2018-04-22        1.524413e+09  1327098000         1.0               0  
2  2018-04-22        1.524413e+09  1379641362         1.0               0  
3  2018-04-22        1.524413e+09  1379641362         1.0               0  
4  2018-04-22        1.524413e+09  1327098000         1.0               0  
5  2018-04-22        1.524413e+09  1327098000         1.0               0  
6  2018-04-22        1.524413e+09  1398108348         1.0               0  
7  2018-04-22        1.524413e+09  1398108348         1.0               0  
8  2018-04-22        1.524413e+09  1398108348         1.0               0  
9  2018-04-22        1.524413e+09  1398108348         1.0               0

In [73]:
final_output = final_output.orderBy("traceTimeentryUnix", ascending=True)
final_output.limit(10).toPandas()

uuid   account  datasetScope  \
0  8d160961f13a4effbc0a381a6518d5ca  tofitsch    mc15_14TeV   
1  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
2  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
3  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
4  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
5  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
6  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
7  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
8  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   
9  895eaa69307d444d91d1d38a26629f83      heng  data17_13TeV   

                                             dataset     fileScope  \
0  mc15_14TeV.167796.Sherpa_CT10_WtaunuMassiveCBP...    mc15_14TeV   
1  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
2  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
3  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
4  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
5  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
6  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
7  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
8  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   
9  data17_13TeV.00338377.physics_Main.deriv.DAOD_...  data17_13TeV   

                                    filename eventType         clientState  \
0   DAOD_TRUTH3.13422301._000001.pool.root.1  download                DONE   
1  DAOD_HIGG5D1.12819918._000580.pool.root.1  download  ServiceUnavailable   
2  DAOD_HIGG5D1.12819918._000581.pool.root.1  download  ServiceUnavailable   
3  DAOD_HIGG5D1.12819918._000581.pool.root.1  download  ServiceUnavailable   
4  DAOD_HIGG5D1.12819918._000580.pool.root.1  download  ServiceUnavailable   
5  DAOD_HIGG5D1.12819918._000580.pool.root.1  download  ServiceUnavailable   
6  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   
7  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   
8  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   
9  DAOD_HIGG5D1.12819918._000582.pool.root.1  download  ServiceUnavailable   

          day  traceTimeentryUnix    filesize  prediction  int_percentage  
0  2018-04-22        1.524413e+09   236600961         0.0             100  
1  2018-04-22        1.524413e+09  1327098000         1.0               0  
2  2018-04-22        1.524413e+09  1379641362         1.0               0  
3  2018-04-22        1.524413e+09  1379641362         1.0               0  
4  2018-04-22        1.524413e+09  1327098000         1.0               0  
5  2018-04-22        1.524413e+09  1327098000         1.0               0  
6  2018-04-22        1.524413e+09  1398108348         1.0               0  
7  2018-04-22        1.524413e+09  1398108348         1.0               0  
8  2018-04-22        1.524413e+09  1398108348         1.0               0  
9  2018-04-22        1.524413e+09  1398108348         1.0               0

## get complete amount of data downloaded in time window to calculate good cache size

In [76]:
final_output.createOrReplaceTempView('output')
spark.sql("SELECT SUM(filesize) FROM (SELECT DISTINCT fileScope, fileName, filesize FROM output)").toPandas()

sum(filesize)
0  36398774222862

In [75]:
final_output.write.csv('/user/sprenner/forest_april_official_proportion_predictions_hourly_ordered_numTrees300_seed1234_cache01.csv', sep='\t')

## predict downloads for may with trained model

In [81]:
df_mai = spark.read.json("/user/sprenner/training_official_mai.json")

In [82]:
df_mai.describe().toPandas()

summary   account           accountHash     clientState  \
0   count   5316524               5316524         5316524   
1    mean      None   5.676034578304038E7            None   
2  stddev      None  1.0782381847434077E9            None   
3     min  aalfonsi           -2144719344  AttributeError   
4     max    zzheng            2143592175     SystemError   

        clientStateHash                                            dataset  \
0               5316524                                            5316524   
1  -8.556539364422513E8                                               None   
2   3.393227998953656E8                                               None   
3           -1913538301                                    13917493.events   
4            1269532611  valid2.361023.Pythia8EvtGen_A14NNPDF23LO_jetje...   

           datasetHash datasetScope      datasetScopeHash eventType  \
0              5316524      5316524               5316524   5316524   
1  8.513416665722266E7         None  -2.489378879578484E8      None   
2  1.264352081440324E9         None   1.159227636261917E9      None   
3          -2147477854    cond09_mc           -2145215727  download   
4           2147445885       valid2            2144116684  download   

  eventTypeHash eventVersion       eventVersionHash  \
0       5316524      5316524                5316524   
1    5.937808E7         None  -2.0386347870917916E9   
2           0.0         None   1.2667515763379405E8   
3      59378080       1.13.2            -2044360925   
4      59378080       1.16.1             1352846300   

                                            filename         filenameHash  \
0                                            5316524              5316524   
1                                               None   -576003.5649204254   
2                                               None  1.238741621683768E9   
3                                              0.dat          -2147483353   
4  user.zzheng.410389.MadGraphPythia8EvtGen.DAOD_...           2147479805   

        filesize_double                hits                  hostname  \
0               5316524             5316524                   5316524   
1  2.8167336660135007E8   6.517046664324284                      None   
2  1.0094883391407572E9  13.920048576048558                      None   
3                   2.0                   1    ReCaS-ui-01.cs.infn.it   
4       4.7520668166E10                 249  xenia.nevis.columbia.edu   

           hostnameHash protocol         protocolHash  \
0               5316524  5316524              5316524   
1  1.8503396854168174E8     None  5.671483465013922E8   
2  1.1133988987325292E9     None  9.234807216961868E8   
3           -2140594599     davs          -1607963002   
4            2130161317      srm           2128034789   

                 remoteSite        remoteSiteHash      scope  \
0                   5316524               5316524    5316524   
1                      None  2.0771631249619377E8       None   
2                      None   1.243718578580587E9       None   
3           AGLT2_CALIBDISK           -2113133296  cond09_mc   
4  ZA-WITS-CORE_SCRATCHDISK            2141740830     valid2   

               scopeHash                           traceId  \
0                5316524                           5316524   
1  -2.4953820189199993E8                          Infinity   
2      1.1593529202301E9                               NaN   
3            -2145215727  00000696ec754c998f70d84d737dfd18   
4             2144116684  fffffe65b9c549e5ae746c15ec98d5bb   

            traceIdHash                    traceIp            traceIpHash  \
0               5316524                    5316524                5316524   
1   -254286.60280984343                       None  -4.1963757976758495E7   
2  1.2400714422504258E9                       None   1.2010453269587853E9   
3           -2147483583  2001:1458:201:e4::100:28c            -2143850866   
4            2147482

In [83]:
assembler_mai = VectorAssembler(
    inputCols=["traceTimeentryUnix", "transferStart", "accountHash","clientStateHash","datasetHash", "datasetScopeHash",
               "eventTypeHash", "eventVersionHash","filenameHash", "hostnameHash", "protocolHash", "remoteSiteHash", 
               "scopeHash", "traceIdHash", "traceIpHash", "uuidHash", "filesize_double",
               "projectHash", "run_numberHash", "stream_nameHash", "prod_stepHash", "datatypeHash",
               "dataset_versionHash"],
    outputCol="features")

output_mai = assembler_mai.transform(df_mai)

In [84]:
predicted_mai = model.transform(output_mai)

In [85]:
predicted_mai=predicted_mai.withColumn("filesize", predicted_mai["filesize_double"].cast(IntegerType()))

In [86]:
predicted_mai = predicted_mai.drop('filesize_double')

## save separately for plotting

In [87]:
df_mai_plot = predicted_mai.select('hits', 'prediction')
df_mai_plot.write.csv('/user/sprenner/forest_plot_mai_official_predictions_numTrees300_seed1234_cache01.csv', sep='\t')

In [88]:
df_mai_sim = predicted_mai.select('uuid', 'account', 'datasetScope', 'dataset', 'fileScope','filename', 'eventType', 'clientState', 
                                  from_unixtime(predicted_mai.traceTimeentryUnix, format='yyyy-MM-dd').alias('day'), 'traceTimeentryUnix', 
                                  'filesize', 'prediction', 'int_percentage').sort(col("day").asc())

In [89]:
df_mai_sim.limit(5).toPandas()

uuid account  \
0  c2bb4c50d50f474d82b8eef0a4aaf02a   cylin   
1  c2bb4c50d50f474d82b8eef0a4aaf02a   cylin   
2  c2bb4c50d50f474d82b8eef0a4aaf02a   cylin   
3  a71a723e9f9341c5a18bbdc09b4ee87e   cylin   
4  c2bb4c50d50f474d82b8eef0a4aaf02a   cylin   

                                             dataset  \
0  mc16_13TeV.361024.Pythia8EvtGen_A14NNPDF23LO_j...   
1  mc16_13TeV.361024.Pythia8EvtGen_A14NNPDF23LO_j...   
2  mc16_13TeV.361024.Pythia8EvtGen_A14NNPDF23LO_j...   
3  mc16_13TeV.361021.Pythia8EvtGen_A14NNPDF23LO_j...   
4  mc16_13TeV.361024.Pythia8EvtGen_A14NNPDF23LO_j...   

                           filename eventType         clientState         day  \
0  AOD.12911516._000058.pool.root.1  download  ServiceUnavailable  2018-05-01   
1  AOD.12911516._000058.pool.root.1  download  ServiceUnavailable  2018-05-01   
2  AOD.12911516._000058.pool.root.1  download  ServiceUnavailable  2018-05-01   
3  AOD.12911492._000332.pool.root.1  download  ServiceUnavailable  2018-05-01   
4  AOD.12911516._000058.pool.root.1  download  ServiceUnavailable  2018-05-01   

   traceTimeentryUnix    filesize  prediction  
0        1.525170e+09  2147483647         1.0  
1        1.525170e+09  2147483647         1.0  
2        1.525170e+09  2147483647         1.0  
3        1.525170e+09  2147483647         1.0  
4        1.525170e+09  2147483647         1.0

In [90]:
#df_mai_sim.write.csv('/user/sprenner/forest_mai_official_predictions_numTrees300_seed1234_cache01.csv', sep='\t')